In [ ]:
# optional, only for Jupyter
%matplotlib notebook

# General libraries
import numpy as np                # to deal with arrays, vectors, matrices...
import matplotlib.pyplot as plt   # to plot the data
import matplotlib.dates as mdates
import pandas as pd
import matplotlib.gridspec as gridspec
from tqdm import tqdm

# Tensorflow
import os
HOME = os.getenv('HOME')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to get rid of the TF compilation warnings
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, LSTM, Reshape, Flatten
from tensorflow.keras.utils import get_file
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Only because my system-wide config is tuned, you don't need these lines
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 5,3
mpl.rcParams['font.size'] = 6.0

# The Data

In [ ]:
# Get the data
url = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip'
csv_path = f'{HOME}/tensorflow_datasets/climate/'
csv_path += 'jena_climate_2009_2016.csv.zip'

zip_path = get_file(origin = url, fname = csv_path,
                    archive_format='zip',extract=True)

Read the data and fix format if necessary

In [ ]:
df0 = pd.read_csv(csv_path)

# Convert dates & sort
df0['Date Time'] = pd.to_datetime(df0['Date Time'],format='%d.%m.%Y %H:%M:%S')
df0 = df0.sort_values(by='Date Time')

print(len(df0.index.values))

## Explore the data

In [ ]:
fig = plt.figure(figsize=(7,12))
n = len(df0.columns[1:])
gs = gridspec.GridSpec(n, 1)
axs = []
for i in range(n):
    if i == 0:
        axs.append( fig.add_subplot(gs[i, 0]) )
    else:
        axs.append( fig.add_subplot(gs[i, 0],sharex=axs[0]) )

for i,col in enumerate(df0.columns[1:]):
    axs[i].plot(df0['Date Time'], df0[col])
    axs[i].set_xlabel('DateTime')
    axs[i].set_ylabel(col)
fig.tight_layout()
plt.sho()

In [ ]:
# Registers every 10 min, let's consider one register per hour (RAM, CPU...)
df0 = df0.loc[df0['Date Time'].apply(lambda x: x.minute) == 0]

# Only 1 register for 2017, we skip it
df0 = df0.loc[df0['Date Time'].apply(lambda x: x.year) < 2017]

T0 = None
if not T0 is None:
    T0 = dt.datetime(2015,1,1)
    df = df[df['Date Time'] > T0]

Nsamples = len(df0.index.values)
print(Nsamples)

In [ ]:
fig, ax = plt.subplots()
X = [x.replace(year=2020) for x in df0['Date Time']]
Y = df0['T (degC)']
C = [x.year for x in df0['Date Time']]

img = ax.scatter(X, Y, c=C, cmap='cool')
cbar = fig.colorbar(img) #, orientation='horizontal', shrink=0.8)

cbar.ax.set_ylabel('Year')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%d'))
ax.set_title('By Year')
ax.set_ylabel('T (°C)')
plt.show()

In [ ]:
fig, ax = plt.subplots()
X = [x.time() for x in df0['Date Time']]
C = [x.month for x in df0['Date Time']]
Y = df0['T (degC)']

img = ax.scatter(X, Y, c=C, cmap='cool')
cbar = fig.colorbar(img) #, orientation='horizontal', shrink=0.8)

cbar.ax.set_ylabel('Month')
ax.set_title('By Month')
ax.set_ylabel('T (°C)')
plt.show()

## Useful functions to interact with data

In [ ]:
def visualize_sample(DF,ind,Nhistory,Nfuture,col='T (degC)',pred=None):
    try:
        X0 = DF.iloc[ind-Nhistory:ind]['Date Time']
        X1 = DF.iloc[ind:ind+Nfuture]['Date Time']
        fmt = '%-d/%-m/%y\n%H:%M'
    except KeyError:
        X0 = range(ind-Nhistory,ind)
        x0 = len(X0)
        X1 = range(ind,ind+Nfuture)
        fmt = ''
    Y0 = DF.iloc[ind-Nhistory:ind][col]
    Y1 = DF.iloc[ind:ind+Nfuture][col]
    fig, ax = plt.subplots()
    ax.plot(X0,Y0, 'C0', label='input')
    ax.plot(X1,Y1, 'C1', label='output')
    if pred is not None:
        ax.plot(X1,pred, 'C2', label='NN pred')
    if len(fmt) > 0:
        # there are dates
        X1_base = pd.date_range(min(X1),max(X1))
        Y1_base = base_line(df_valid,min(X1),max(X1),cols=['T (degC)'])
        ax.plot(X1_base,Y1_base, 'C3', label='BaseLine')
        ax.xaxis.set_major_formatter(mdates.DateFormatter(fmt))
    ax.legend()

In [ ]:
def base_line(DF,start,end,cols=['T (degC)']):
    aux = DF.copy()
    aux = aux.groupby([aux['Date Time'].apply(lambda x: x.month),
                       aux['Date Time'].apply(lambda x: x.day)]).mean()
    ret = []
    for date in pd.date_range(start, end):
        ret.append(aux.loc[date.month].loc[date.day][cols].values)
    return np.array(ret)

In [ ]:
# Normalize!!
norm = True
if norm:
    cols = df0.columns.values[1:]   # skip date
    df_num = df0[cols]

    mean = df_num.mean()
    std = df_num.std()

    df = df0.copy()
    df[cols] = (df[cols]-mean)/std
else:
    df = df0

# Prepare Input/Output data

We will use the weather of the past 5 days to predict the weather of the next 12 hours.
The data is collected every 10 min (with some exceptions that we are going to ignore), that means that we need:

Input: $5\text{days}\times 24\text{hours}\times 60\text{hours}/10\text{min between samples} = 720\text{samples}$

Output: $0.5\text{days}\times24\text{hours}\times60\text{hours} / 10\text{min between samples} = 72 \text{samples}$
<img src='inp_out.svg'>

In [ ]:
#df_train = df.loc[df['Date Time'].apply(lambda x:x.year)<2016]
#df_valid = df.loc[df['Date Time'].apply(lambda x:x.year)>=2016]

msk = np.random.rand(len(df)) < 0.8

df_train = df[msk]
df_valid = df[~msk]

In [ ]:
def prepare_inp_out(DF,Nhistory,Nfuture,columns_in,columns_out,step=6):
    IN,OUT = [],[]
    for ind in tqdm(range(Nhistory,len(DF.index)-Nfuture)):
        indices = range(ind-Nhistory, ind, step)
        inp = DF.iloc[indices][columns_in].values
        out = DF.iloc[ind:ind+Nfuture][columns_out].values
        IN.append(inp)
        OUT.append(out)
    return np.array(IN),np.array(OUT)

In [ ]:
Nhistory = int(5*24*60/10)
Nfuture  = int(6*60/10)
# columns_in  = ['T (degC)']
columns_in  = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']
columns_out = ['T (degC)']

x_train,y_train = prepare_inp_out(df_train,Nhistory,Nfuture,columns_in,columns_out)
inp_shape = x_train.shape[1:]
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_valid,y_valid = prepare_inp_out(df_valid,Nhistory,Nfuture,columns_in,columns_out)
print(x_valid.shape)
print(y_valid.shape)

In [ ]:
visualize_sample(df_valid,Nhistory+1,Nhistory,Nfuture,col='T (degC)',pred=None)


In [ ]:
Dataset = tf.data.Dataset
BATCH_SIZE = 256
BUFFER_SIZE = 10000

In [ ]:
train_data = Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

In [ ]:
valid_data = Dataset.from_tensor_slices((x_valid, y_valid))
valid_data = valid_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

## Define the Model

In [ ]:
model = models.Sequential()
model.add(LSTM(32, return_sequences=True, input_shape=inp_shape))
model.add(tf.keras.layers.LSTM(16, activation='relu'))
model.add( Dense(100, activation='relu') )
model.add( Dense(Nfuture*len(columns_out),activation='linear') )
model.add( Reshape((Nfuture,len(columns_out))) )

model.summary()

In [ ]:
model.compile(optimizer=RMSprop(clipvalue=1.0), loss='mae')

In [ ]:
history = model.fit(train_data,
                    epochs=50,
                    steps_per_epoch=200,
                    validation_data=valid_data,
                    validation_steps=50,
                    verbose=1)

In [ ]:
err = history.history['loss']
val_err = history.history['val_loss']

fig, ax = plt.subplots()
ax.plot(err,label='loss')
ax.plot(val_err,label='val_los')
ax.legend()
ax.set_ylim([0,min([20,np.max(val_err)])])
plt.show()

### Check unseen examples

In [ ]:
outs = model.predict(x_valid)
print(outs.shape)
print(model.evaluate(x_valid,y_valid))

In [ ]:
for _ in range(3):
    ind = np.random.randint(0,len(outs))
    visualize_sample(df_valid,ind,Nhistory,Nfuture,col='T (degC)',pred=outs[ind])
